In [1]:
import pandas as pd
import numpy as np

import geopandas as gpd
import geopy

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter


In [2]:
df = pd.read_excel("output/distribucion_sociodemografica/distribucion_polars.xlsx")

In [3]:
locator = Nominatim(user_agent="javier", timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)

def location_info(x):
    try:
        data = locator.geocode(x).raw
        data_converted = pd.json_normalize(data).squeeze()

    except:
        data_converted = pd.Series(dtype='float64')

    return data_converted


In [5]:
regiones = list(df["region_pais"].unique())
regiones_unicas = pd.DataFrame(regiones, columns=["region_pais"])
objeto = regiones_unicas["region_pais"].apply(location_info)
objeto["lat"] = objeto["lat"].astype(float)
objeto["lon"] = objeto["lon"].astype(float)

union = pd.concat([regiones_unicas, objeto], axis=1)

In [6]:
union

,region_pais,place_id,licence,osm_type,osm_id,boundingbox,lat,lon,display_name,class,type,importance,icon
0,"Región Metropolitana de Santiago, Chile",297794908.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,198848.0,"[-34.292002, -32.9221679, -71.7148502, -69.770...",-33.573934,-70.620552,"Región Metropolitana de Santiago, Chile",boundary,administrative,1.034079,https://nominatim.openstreetmap.org/ui/mapicon...
1,"Región De Ñuble, Chile",298309976.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,7421025.0,"[-37.1971798, -36.0053654, -72.8857814, -71.00...",-36.633158,-71.938482,"Región de Ñuble, Chile",boundary,administrative,0.829042,https://nominatim.openstreetmap.org/ui/mapicon...
2,"Región De Valparaíso, Chile",297881447.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,198847.0,"[-33.9561939, -26.2862267, -109.4548826, -69.9...",-32.597609,-70.852975,"Región de Valparaíso, Chile",boundary,administrative,0.942216,https://nominatim.openstreetmap.org/ui/mapicon...
3,Región Aysén del General Carlos Ibáñez del Cam...,297627342.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,305693.0,"[-49.2115311, -43.5859865, -75.7003818, -71.09...",-46.471156,-73.520671,Región Aysén del General Carlos Ibáñez del Cam...,boundary,administrative,1.460010,https://nominatim.openstreetmap.org/ui/mapicon...
4,"Región Del Maule, Chile",297192947.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,239882.0,"[-36.5436197, -34.684525, -72.7863763, -70.307...",-35.597228,-71.488680,"Región del Maule, Chile",boundary,administrative,0.921505,https://nominatim.openstreetmap.org/ui/mapicon...
5,Región del Libertador General Bernardo O'Higgi...,297867902.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,206487.0,"[-35.0059157, -33.850566, -72.0559225, -70.009...",-34.534538,-71.035482,Región del Libertador General Bernardo O'Higgi...,boundary,administrative,1.320031,https://nominatim.openstreetmap.org/ui/mapicon...
6,"Región De Tarapacá, Chile",297444843.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,238393.0,"[-21.6277488, -18.9365183, -70.2862232, -68.40...",-20.163667,-69.546345,"Región de Tarapacá, Chile",boundary,administrative,0.911311,https://nominatim.openstreetmap.org/ui/mapicon...
7,"Región De La Araucanía, Chile",297612420.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,296378.0,"[-39.6372157, -37.5817959, -73.5190728, -70.82...",-38.667896,-72.261007,"Región de la Araucanía, Chile",boundary,administrative,1.006868,https://nominatim.openstreetmap.org/ui/mapicon...
8,"Región De Atacama, Chile",297960105.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,271889.0,"[-29.5347206, -25.2868441, -71.5925252, -68.26...",-27.557178,-70.015688,"Región de Atacama, Chile",boundary,administrative,0.941156,https://nominatim.openstreetmap.org/ui/mapicon...
9,"Región De Antofagasta, Chile",297933500.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,240932.0,"[-26.0610992, -20.9343862, -70.7371393, -66.99...",-23.604131,-69.084278,"Región de Antofagasta, Chile",boundary,administrative,0.934614,https://nominatim.openstreetmap.org/ui/mapicon...


In [16]:
data_converted

place_id                                                298083588
licence         Data © OpenStreetMap contributors, ODbL 1.0. h...
osm_type                                                 relation
osm_id                                                     305693
boundingbox     [-49.2115311, -43.5859865, -75.7003818, -71.09...
lat                                                   -46.4711556
lon                                                   -73.5206714
display_name    Región Aysén del General Carlos Ibáñez del Cam...
class                                                    boundary
type                                               administrative
importance                                                1.46001
icon            https://nominatim.openstreetmap.org/ui/mapicon...
Name: 0, dtype: object

In [5]:
comunas = list(df["comuna_region_pais"].unique())
comunas_unicas = pd.DataFrame(comunas, columns=["comuna_region_pais"])
objeto_comunas = comunas_unicas['comuna_region_pais'].apply(location_info)
union_comunas = pd.concat([comunas_unicas, objeto_comunas], axis=1)

: 

In [78]:
with pd.ExcelWriter('localizacion.xlsx') as f:
    union.to_excel(f, sheet_name='regiones', index=False)
    union_comunas.to_excel(f, sheet_name='comunas', index=False)

In [ ]:
union